In [ ]:
import Pkg
Pkg.activate()
using FUSE, IMAS, QED, FRESCO, VacuumFields
using LinearAlgebra
using Interact, Plots

In [ ]:
recycle = true
if recycle
    dd = IMAS.json2imas("./dd_d3d.json")
else
    dd, ini, act = FUSE.init(:D3D);
    act.ActorEquilibrium.model = :FRESCO
    FUSE.ActorStationaryPlasma(dd, act);
    IMAS.imas2json(dd, "./dd_d3d.json");
end;

In [ ]:
canvas0 = FRESCO.Canvas(dd, 65);
profile0 = FRESCO.PressureJtoR(dd);
eqt = dd.equilibrium.time_slice[];
cp1d = dd.core_profiles.profiles_1d[];
Q0 = FUSE.qed_init_from_imas(eqt, cp1d; uniform_rho = 101);
η0 = FUSE.η_imas(cp1d);

In [ ]:
canvas, profile = deepcopy(canvas0), deepcopy(profile0);
control = :shape
@time FRESCO.solve!(canvas, profile, 100, 3; control, relax=0.25, tolerance=1e-8, debug=true, initialize_current=true);

In [ ]:
canvas_save = deepcopy(canvas);
profile_save = deepcopy(profile);
dd_save = deepcopy(dd);
Q_save = deepcopy(Q0);

In [ ]:
plot(canvas, size=(400,600), plot_control_points=false, plot_target_boundary=true)

In [ ]:
# need eta as a function of psin
# pressure, thus Te, thus eta held fix wrt psi in equilibrium
cp1d = dd.core_profiles.profiles_1d[]
psin = cp1d.grid.psi_norm
η = QED.η_FE(psin, 1.0 ./ cp1d.conductivity_parallel; use_log=true);

In [ ]:
# GOOD - BENCHMARKED
function Wp(canvas)
    Rs, Zs, Ψ, Ψbnd, Jt, is_inside = canvas.Rs, canvas.Zs, canvas.Ψ, canvas.Ψbnd, canvas._Jt, canvas._is_inside
    Wp = sum(is_inside[k] ? (Ψ[k] - Ψbnd) * Jt[k] : 0.0 for k in eachindex(Jt))
    return -0.5 * Base.step(Rs) * Base.step(Zs) * Wp
end

In [ ]:
FRESCO.sync_Ψ!(canvas; update_vacuum=true, update_Ψitp=true)

Zm, Rm = IMAS.meshgrid(canvas.Zs, canvas.Rs)
Br, Bz = IMAS.Br_Bz(canvas._Ψitp, Rm, Zm)

# 2πR factor for volume
W_from_B2 = 2π * sum(canvas._is_inside .* Rm .* (Br .^ 2 + Bz .^ 2)) * Base.step(canvas.Rs) * Base.step(canvas.Zs) / (8e-7 * π)
@show Wp(canvas), W_from_B2

# From the dd
PolygonOps = IMAS.PolygonOps
eqt = dd.equilibrium.time_slice[]
eq1d = eqt.profiles_1d
eq2d = eqt.profiles_2d[1]

pr, pz = eqt.boundary.outline.r, eqt.boundary.outline.z
bnd = collect(zip(pr, pz))
incore(r, z, bnd) = PolygonOps.inpolygon((r, z), bnd)

dr = eq2d.r[2,1] - eq2d.r[1,1]
dz = eq2d.z[1,2] - eq2d.z[1,1]
W1 = 2π * sum(incore.(eq2d.r, eq2d.z, Ref(bnd)) .* eq2d.r .*  (eq2d.b_field_r .^ 2 + eq2d.b_field_z .^ 2)) * dr * dz / (8e-7 * π)
W2 = -0.5 * sum(incore.(eq2d.r, eq2d.z, Ref(bnd)) .* (eq2d.psi .- eq1d.psi[end]) .* eq2d.j_tor) * dr * dz
@show W1, W2, Wp(canvas);


In [ ]:
function Li(canvas)
    return 2.0 * Wp(canvas) / canvas.Ip ^2
end
Li(canvas, profile)

In [ ]:
function Rp(canvas, eta)
    Rs, Zs, Ψ, Ψaxis, Ψbnd = canvas.Rs, canvas.Zs, canvas.Ψ, canvas.Ψaxis, canvas.Ψbnd
    Ip, Jt, is_inside =  canvas.Ip, canvas._Jt, canvas._is_inside
    inv_ΔΨ = 1.0 / (Ψbnd - Ψaxis)
    psin(psi) = (psi - Ψaxis) * inv_ΔΨ
    Rp = sum(is_inside[k] ? eta(psin(Ψ[k])) * Jt[k] ^ 2 : 0.0 for k in eachindex(Jt))
    return Rp / Ip ^ 2
end
Rp(canvas, η)

In [ ]:
canvas.Fbnd

In [ ]:
Ψaxis, Ψbnd, Fbnd, surfaces = canvas.Ψaxis, canvas.Ψbnd, canvas.Fbnd, canvas._surfaces
psi1d = range(Ψaxis, Ψbnd, length(surfaces))
F2 = 2 * IMAS.cumtrapz(psi1d, FRESCO.ffprime(canvas, profile))
F2 .= F2 .- F2[end] .+ Fbnd^2
F = sign(Fbnd) .* sqrt.(F2)

In [ ]:
active_coils = VacuumFields.MultiCoils(dd)[1:end-2];
passive_coils = VacuumFields.QuadCoil[]
for loop in dd.pf_passive.loop
    for element in loop.element
        outline = element.geometry.outline
        @assert length(outline.r) == 4 "For the time being passive structures must be composed of quadrilateral elements"
        passive_coil = VacuumFields.QuadCoil(outline.r, outline.z)
        passive_coil.resistance = VacuumFields.resistance(passive_coil, loop.resistivity)
        push!(passive_coils, passive_coil)
    end
end
coils = deepcopy(vcat(active_coils, passive_coils));

In [ ]:
# COIL ONLY PROPERTIES
Mcc = [VacuumFields.mutual(c1, c2) for c1 in coils, c2 in coils]

# coil current in this model is current per turn
Ic = [VacuumFields.current(c) / VacuumFields.turns(c) for c in coils]
Rc = [VacuumFields.resistance(c) for c in coils];
Vc = zero(Ic);

In [ ]:
# PLASMA INDUCTANCES

eqt = dd.equilibrium.time_slice[]
cp1d = dd.core_profiles.profiles_1d[]
Ip = eqt.global_quantities.ip

# COIL MUTUALS
image = VacuumFields.Image(eqt)
Mpc = [VacuumFields.mutual(image, coil, Ip) for coil in coils]
dMpc_dt = zero(Mpc) # How Mpc changes in time (like shape)... to test later

# INTERNAL INDUCTANCE
It = IMAS.cumtrapz(cp1d.grid.area, cp1d.j_tor)
Wp = 0.5 * IMAS.trapz(cp1d.grid.psi, It)
Li = 2 * Wp / Ip^2

# EXTERNAL INDUCTANCE 
ψb = eqt.profiles_1d.psi[end]
ψc = sum(Mpc[k] * Ic[k] for k in eachindex(coils))
Le = (ψb - ψc) / Ip
Lp = Li + Le

@show Li, Le, Lp

In [ ]:
# COMPUTE RESISTANCE AS P_OHM / (IP * IOHM)
Pohm = dd.core_sources.source[:ohmic].profiles_1d[].electrons.power_inside[end]
Ini = dd.core_profiles.global_quantities.current_non_inductive[]
Iohm = Ip - Ini
Rp = Pohm / (Ip * Iohm)

In [ ]:
W0 = QED.Waveform{Float64}(t -> 0.0)
W1 = QED.Waveform{Float64}(t -> -1e3) # VERY STRONG 
#Wramp = QED.Waveform{Float64}(t -> 1e3 * (t < t_ramp ? t/t_ramp : 1.0))
V_waveforms = [k <= 6 ? W1 : W0 for k in eachindex(coils)];

In [ ]:
# non_inductive contribution
B0 = eqt.global_quantities.vacuum_toroidal_field.b0
JBni = QED.FE(cp1d.grid.rho_tor_norm, cp1d.j_non_inductive .* B0);

In [ ]:
# Initialize
QI = FUSE.qed_init_from_imas(eqt, cp1d; uniform_rho = 101)
#QI.JBni = nothing;
if QI.JBni === nothing
    Vni = 0.0 
else
    Vni = Rp * Ini
end
build = QED.QED_build(Ic, Vc, Rc, Mcc, Vni, Rp, Lp, Mpc, dMpc_dt, V_waveforms);

In [ ]:
# Plasma only diffusion problem - standard QED

t0 = dd.global_time
t1 = t0 + 5.0
δt = t1 - t0
No = 1
Ni = 1000
Ip = nothing
Vedge = 0.0
time0 = t0 + δt / 2.0
Qd = QED.diffuse(QI, FUSE.η_imas(dd.core_profiles.profiles_1d[time0]), δt, Ni; Vedge, Ip, debug=true);
@show QED.Ip(QI), QED.Ip(Qd)

In [ ]:
# Full coupled evolution
BO = deepcopy(build)
Qe = QED.evolve(deepcopy(QI), FUSE.η_imas(dd.core_profiles.profiles_1d[time0]), BO, δt, Ni; debug=true);
plot(Qd.ρ, QED.Jt_R(Qd), lw=3)
display(plot!(Qe.ρ, QED.Jt_R(Qe), lw=3, ls=:dash))
@show QED.Ip(QI), QED.Ip(Qd), QED.Ip(Qe)

In [ ]:
c = cgrad(:diverging)
B = BO
Jmax = maximum(abs.(B.Ic))
p = plot(aspect_ratio=:equal)
for (i,coil) in enumerate(coils)
    current_color_index = (B.Ic[i] + Jmax) / (2 * Jmax)
    plot!(p, coil,color=c[current_color_index], alpha=1.0)
end
display(p)

In [ ]:
Lp

In [ ]:
dd.equilibrium.time

In [ ]:
Rp

In [ ]:
Lp

In [ ]:
IMAS.imas2json(dd, "/Users/lyons/Downloads/iter_ods.json", freeze=true, strict=true);

In [ ]:
[VacuumFields.turns(c) for c in active_coils]

In [ ]:
dd.pf_active.coil[1]

In [ ]:
Ic = [VacuumFields.current(c) for c in dd.pf_active.coil]
@show Ic

In [ ]:
(-2.326557198014836e7 -1.8934747387408897e7) /2

In [ ]:
R = collect(range(3,9,65))
Z = collect(range(-6,6,129))
FUSE.ActorTEQUILA(dd, act; ip_from=:core_profiles, R, Z)

In [ ]:
eq2d = dd.equilibrium.time_slice[].profiles_2d[2]
R = eq2d.dim1
Z = eq2d.dim2
Jt = eq2d.j_tor
psi2d = 

In [ ]:
IMAS.imas2json(dd, "/Users/lyons/Downloads/iter_ods2.json", freeze=true, strict=true);